<a href="https://colab.research.google.com/github/thainazanfolin/min_sleep_efficiency/blob/main/Projeto_Min_Sleep_Efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Projeto de Análise de dados: como os fatores de Estilo de Vida influenciam a Eficiência do Sono

- [Link do dataset no Kaggle](https://www.kaggle.com/datasets/equilibriumm/sleep-efficiency/data)

## Objetivo do projeto
Realizar uma análise sobre como diferentes hábitos podem ou não afetar a eficiência do sono, considerando tempo de sono e também sua qualidade.

## Sobre o Conjunto de Dados
O conjunto de dados contém informações sobre um grupo de sujeitos de teste e seus padrões de sono.
- **Subject ID**: Identificação exclusiva de cada sujeito, com idade e gênero registrados.
- **Bedtime**: Hora em que cada sujeito vai para a cama.
- **Wakeup time**: Hora em que cada sujeito acorda.
- **Sleep duration**: Total de tempo dormido em horas.
- **Sleep efficiency**: Proporção do tempo na cama que foi realmente gasto dormindo.
- **REM sleep percentage**: Tempo gasto em sono REM.
- **Deep sleep percentage**: Tempo gasto em sono profundo.
- **Light sleep percentage**: Tempo gasto em sono leve.
- **Awakenings**: Número de vezes que cada sujeito acorda durante a noite.
- **Caffeine consumption**: Consumo de cafeína nas 24 horas anteriores à hora de dormir.
- **Alcohol consumption**: Consumo de álcool nas 24 horas anteriores à hora de dormir.
- **Smoking status**: Estado de fumante de cada sujeito.
- **Exercise frequency**: Frequência de exercícios de cada sujeito.



## Vamos importar a nossa base de dados.
 Optamos por trazer por URL para facilitar o trabalho de todos os membros da equipe em um só colab. <br> Porém, também pode ser feito subindo a base aqui no Colab, normalmente. <br><br>
Vamos começar!

In [4]:
import pandas as pd

url = 'https://raw.githubusercontent.com/thainazanfolin/min_sleep_efficiency/refs/heads/main/Sleep_Efficiency_table.csv'
df = pd.read_csv(url, delimiter=';')

# Verifique as primeiras linhas do DataFrame
print(df.head(100))


     ID  Age  Gender           Bedtime       Wakeup time  Sleep duration  \
0     1   65  Female  06/03/2021 01:00  06/03/2021 07:00             6.0   
1     2   69    Male  05/12/2021 02:00  05/12/2021 09:00             7.0   
2     3   40  Female  25/05/2021 21:30  25/05/2021 05:30             8.0   
3     4   40  Female  03/11/2021 02:30  03/11/2021 08:30             6.0   
4     5   57    Male  13/03/2021 01:00  13/03/2021 09:00             8.0   
..  ...  ...     ...               ...               ...             ...   
95   96   61  Female  10/07/2021 02:00  10/07/2021 09:00             7.0   
96   97   41    Male  15/02/2021 21:00  15/02/2021 04:00             7.0   
97   98   53    Male  02/11/2021 23:00  02/11/2021 06:30             7.5   
98   99   32  Female  12/01/2021 02:00  12/01/2021 11:00             9.0   
99  100   65    Male  07/12/2021 01:30  07/12/2021 09:30             8.0   

    Sleep efficiency  REM sleep percentage  Deep sleep percentage  \
0               0.